In [1]:
import re
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# set better display
pd.set_option('display.expand_frame_repr', False)

In [2]:
# import the SeeClickFix issues and comments csvs as dataframes
issues_df = pd.read_csv('all_scf_issues.csv', encoding="latin1", low_memory=False)
comments_df = pd.read_csv('scf_issues_with_comments.csv', encoding="latin1", low_memory=False)

# check that dataframe dimensions (rows, cols) are as expected
print(issues_df.shape, comments_df.shape)

(16431, 43) (14938, 5)


In [3]:
# define dict of Service Level Agreements by type with days committed to close
sla_days_to_close = {
    'Abandoned Vehicle': 5,
    'Blocked Catch Basin': 1,
    'Curbside Solid Waste Issue': 7,
    'DPW - Debris Removal - DPW USE ONLY': 0,
    'DPW - Other environmental': 0,
    'Dead Animal Removal': 3,
    'Fire Hydrant Issue': 1,
    'Illegal Dump Sites': 10,
    'Illegal Dumping In Progress': 7,
    'Manhole Cover Issue': 1,
    'New LED Street Light Out': 7,
    'Park Issue': 5,
    'Potholes': 5,
    'Residential Snow Removal Issue': 1,
    'Rodent Extermination - DPW Only': 0,
    'Running Water in a Home or Building': 4,
    'Street Light Pole Down': 2,
    'Traffic Sign Issue': 7,
    'Traffic Signal Issue': 2,
    'Tree Issue': 14,
    'Water Main Break': 1 
}

# transform the SLA dict to a dataframe
sla_df = pd.DataFrame(list(sla_days_to_close.items()), columns=['request_type_title', 'sla_days_to_close'])

In [4]:
# join issues to comments
comments_df = comments_df[['id','comment_type_issue_closed']]
scf_df = pd.merge(issues_df, comments_df, on="id", how="left")

# replace NaN comments with empty string to be able to use regex
scf_df['comment_type_issue_closed'] = scf_df['comment_type_issue_closed'].fillna('')

# preview df
scf_df[:3]

,id,status,summary,description,rating,lat,lng,address,created_at,acknowledged_at,...,reporter_id,reporter_name,reporter_role,agent_id,agent_name,agent_role,request_type_id,request_type_title,canonical_issue_id,comment_type_issue_closed
0,3126258,Archived,New LED Street Light Out,New historic replica light on the northwest co...,2,42.356047,-83.067835,"410 West Warren Avenue Detroit, Michigan",2017-01-01T04:01:42-05:00,2017-01-03T07:57:52-05:00,...,1207068,Jefe,Registered User,1207068,Jefe,Registered User,16517,New LED Street Light Out,NaN,Thank you for reporting this issue to the City...
1,3126259,Archived,Traffic Sign Issue,Pedestrian crossing sign in the northbound lan...,3,42.360299,-83.068776,"5470 Cass Avenue Detroit, Michigan",2017-01-01T04:10:34-05:00,2017-01-03T06:59:33-05:00,...,1207068,Jefe,Registered User,1207068,Jefe,Registered User,9123,Traffic Sign Issue,NaN,This message is to inform you that the issue y...
2,3126277,Acknowledged,Water Main Break,Frozen water about 2 blocks,2,42.346605,-83.137069,"6659 Floyd Avenue Detroit, Michigan",2017-01-01T07:18:39-05:00,2017-01-01T07:25:21-05:00,...,1314738,Geretha Ridley,Registered User,1314738,Geretha Ridley,Registered User,8247,Water Main Break,NaN,


In [5]:
# define list of key words that indicate issues are out of jurisdiction
keywords = [
    "jurisdiction",
    "Jurisdiction",
    "unable to locate",
    "invalid",
    "DTE",
    "referred",
    "Detroit Land Bank",
    "Belle Isle",
    "Wayne",
    "State",
    "MDOT"]

In [6]:
# HELPER FUNCTIONS

def getDate(obj):
    """ Reformats object as mm-dd-yyyy """
    return datetime.strptime(obj[:-6], '%Y-%m-%dT%H:%M:%S').strftime('%m-%d-%Y')

def makeFloat(td):
    """ Converts a timedelta to a float, eg 2 days 12:00:00 becomes 2.5 """
    return td.total_seconds() / timedelta (days=1).total_seconds()

def slaCheck(row):
    """ Checks if issues are closed within their SLA, returns 1 if yes/under, 0 if no/over, or NaN if not yet closed """
    if (row['days_create_to_close'] > 0):
        if row['days_create_to_close'] < sla_days_to_close[row['request_type_title']]:
            return 1
        return 0
    pass

def trimTimezone(obj):
    """ Trims timezone off end of datetime """
    return datetime.strptime(obj[:-6], '%Y-%m-%dT%H:%M:%S')

def jurisdictionCheck(string):
    """ Checks if issues are closed because they're out of City jurisdiction based on comment text, returns 1 if yes """
    found_keywords = re.findall(r"(?=("+'|'.join(keywords)+r"))", string)
    if len(found_keywords) > 0:
        return 1
    elif len(found_keywords) < 1:
        return 0
    else:
        pass

In [7]:
# NEW COLUMNS

# add 1 new col, returns mm-dd-yyyy object
scf_df['created_at_date'] = scf_df['created_at'].apply(lambda x: getDate(x))

# add 2 new cols, transforms an object to datetime64[ns] type
scf_df['created_at_datetime'] = pd.to_datetime(scf_df['created_at'])
scf_df['closed_at_datetime'] = pd.to_datetime(scf_df['closed_at'])

# add 1 new col with trimmed time to index by
scf_df['time_index'] = scf_df['created_at'].apply(lambda x: trimTimezone(x))

# add 1 new col with diff value, returns timedelta or NaT if the issue is not closed yet
scf_df['diff_create_to_close'] = scf_df['closed_at_datetime'] - scf_df['created_at_datetime']

# add 1 new col with fractional days, returns float or NaN if the issue is not closed yet
scf_df['days_create_to_close'] = scf_df['diff_create_to_close'].apply(lambda x: makeFloat(x))

# add 1 new col, indicates whether a closed issue is within its SLA
scf_df['within_sla_bool'] = scf_df.apply(lambda row: slaCheck(row), axis=1)

# add 1 new col, indicates whether issue was closed because beyond jurisdiction
scf_df['beyond_jurisdiction'] = scf_df['comment_type_issue_closed'].apply(lambda x: jurisdictionCheck(x))

# check that we successfully added 8 new cols
print(scf_df.shape)

(16438, 52)
